## ARMA time series forecasting

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import sys
sys.path.append('..')

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from statsmodels.tsa.api import VARMAX, SARIMAX

from ts_utils import make_sine_cosine

In [ ]:
np.random.seed(12345)

## Data generation

In [ ]:
num_steps = 300
max_length = 100.
noise_level = 0.1

data = make_sine_cosine(
    num_steps=num_steps,
    max_length=max_length,
    noise_level=noise_level,
    val_size=None
)

print('Data shape:', data.shape)

In [ ]:
df = pd.DataFrame({
    'time': pd.date_range("2026-01-01", periods=len(data), freq="D"),
    'sine': data[:, 0],
    'cosine': data[:, 1]
})

df.head()

In [ ]:
fig, axes = plt.subplots(nrows=2, ncols=1, figsize=(6, 4))
for idx, ax in enumerate(axes.ravel()):
    ax.plot(np.arange(len(data)) + 1, data[:,idx], color=plt.cm.Dark2(idx), alpha=0.7)
    ax.set(xlabel='t', ylabel=f'$y_{idx + 1}$')
    ax.set_xlim((0, len(data)))
    ax.grid(visible=True, which='both', color='lightgray', linestyle='-')
    ax.set_axisbelow(True)
fig.tight_layout()

In [ ]:
val_size = 0.2

train_df, val_df = train_test_split(
    df,
    test_size=val_size,
    shuffle=False
)

train_data = train_df[['sine', 'cosine']].to_numpy()
val_data = val_df[['sine', 'cosine']].to_numpy()

print('Train data shape:', train_data.shape)
print('Val. data shape:', val_data.shape)

## Vector-AR model

In [ ]:
model = VARMAX(
    train_data,
    order=(5, 0),
    trend='n'
    # error_cov_type='diagonal',
    # dates=df['time'],
    # freq="D"
)

In [ ]:
res = model.fit(maxiter=1000)

res.summary()

In [ ]:
predict = res.get_prediction(end=model.nobs + len(val_data))

pred_mean = predict.predicted_mean
pred_ci = predict.conf_int(alpha=0.5)

# val_pred_mean = res.forecast(steps=len(val_data))

In [ ]:
fig, axes = plt.subplots(nrows=2, ncols=1, figsize=(6, 4))
for idx, ax in enumerate(axes.ravel()):
    ax.plot(np.arange(len(train_data)) + 1, train_data[:,idx], color=plt.cm.Dark2(idx), alpha=0.7)
    ax.plot(np.arange(len(pred_mean) - 1) + 2, pred_mean[1:,idx], color=plt.cm.Dark2(idx), alpha=0.7, linestyle='--')
    ax.set(xlabel='t', ylabel=f'$y_{idx + 1}$')
    ax.set_xlim(xmin=0)
    ax.grid(visible=True, which='both', color='lightgray', linestyle='-')
    ax.set_axisbelow(True)
fig.tight_layout()

## Multiple SARIMAX models

In [ ]:
columns = ['sine', 'cosine']

train_dict = {c: train_df[c].to_numpy() for c in columns}
val_dict = {c: val_df[c].to_numpy() for c in columns}

model_dict = {
    c: SARIMAX(
        train_dict[c],
        order=(10, 0, 0),
        trend='n'
    ) for c in columns
}

In [ ]:
res_dict = {c: model.fit(maxiter=1000) for c, model in model_dict.items()}

for c, res in res_dict.items():
    print(res.summary())

In [ ]:
predict_dict = {c: res.get_prediction(end=model_dict[c].nobs + len(val_dict[c])) for c, res in res_dict.items()}

pred_mean_dict = {c: predict.predicted_mean for c, predict in predict_dict.items()}
pred_ci_dict = {c: predict.conf_int(alpha=0.5) for c, predict in predict_dict.items()}

pred_mean = np.stack([pred_mean_dict[c] for c in columns], axis=1)
pred_ci = np.concatenate([pred_ci_dict[c] for c in columns], axis=1)

In [ ]:
fig, axes = plt.subplots(nrows=2, ncols=1, figsize=(6, 4))
for idx, ax in enumerate(axes.ravel()):
    ax.plot(np.arange(len(train_data)) + 1, train_data[:,idx], color=plt.cm.Dark2(idx), alpha=0.7)
    ax.plot(np.arange(len(pred_mean) - 1) + 2, pred_mean[1:,idx], color=plt.cm.Dark2(idx), alpha=0.7, linestyle='--')
    ax.set(xlabel='t', ylabel=f'$y_{idx + 1}$')
    ax.set_xlim(xmin=0)
    ax.grid(visible=True, which='both', color='lightgray', linestyle='-')
    ax.set_axisbelow(True)
fig.tight_layout()